### Script to parse security returns CSV file
#### Does not use sys and command line args

In [301]:
import numpy as np
import pandas as pd
from datetime import datetime

src_csv_name = 'SP500_GSPC_monthly.csv'
snp500 = pd.read_csv(src_csv_name)
snp500d = pd.read_csv('SP500_GSPC_daily.csv')
snp500.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-01,16.660000,17.090000,16.650000,17.049999,17.049999,42570000
1,1950-02-01,17.049999,17.350000,16.990000,17.219999,17.219999,33430000
2,1950-03-01,17.240000,17.610001,17.070000,17.290001,17.290001,40410000
3,1950-04-01,17.340000,18.070000,17.340000,18.070000,18.070000,48250000
4,1950-05-01,18.219999,18.780001,18.110001,18.780001,18.780001,45080000


In [302]:
snp500['Date'] = snp500['Date'].astype('datetime64[ns]')  #to convert to a pandas timestamp object
# snp500['Date'] = [time.date() for time in snp500['Date']] #to convert the pandas timestamp to a datetime.date object

In [305]:
# Uncomment and Rerun the set_index line if the CSV file is reloaded into the dataframe
# snp500.set_index( 'Date', inplace=True)
snp500['Monthly Return'] = ( snp500['Adj Close'].pct_change() ) * 100
snp500.loc['1950-12-01',:]
snp500.head()

,Open,High,Low,Close,Adj Close,Volume,Monthly Return
Date,,,,,,,
1950-01-01,16.660000,17.090000,16.650000,17.049999,17.049999,42570000,NaN
1950-02-01,17.049999,17.350000,16.990000,17.219999,17.219999,33430000,0.997068
1950-03-01,17.240000,17.610001,17.070000,17.290001,17.290001,40410000,0.406516
1950-04-01,17.340000,18.070000,17.340000,18.070000,18.070000,48250000,4.511272
1950-05-01,18.219999,18.780001,18.110001,18.780001,18.780001,45080000,3.929170


In [306]:
snp500['12MMA'] = snp500['Monthly Return'].rolling(window=12).mean()
summary = snp500.describe()
summary
snp500['Monthly Return'].std()

4.10097051671692

In [266]:
# Generating CSV files
snp500.to_csv( (src_csv_name.split('.'))[0]+' calculated.csv')
summary.to_csv( (src_csv_name.split('.'))[0]+' summary.csv')

In [300]:
snp500d['Date'] = snp500d.astype( 'datetime64[ns]')
snp500d.set_index('Date', inplace=True)

price_interval = (snp500d.index[1]-snp500d.index[0])

In [273]:
snp500d.describe()

,Open,High,Low,Close,Adj Close,Volume,Daily Return,10DMA,30DMA,Cumulative Returns
count,17336.000000,17336.000000,17336.000000,17336.000000,17336.000000,1.733600e+04,17335.000000,17326.000000,17306.000000,17335.000000
mean,565.729369,569.063398,562.198907,565.846731,565.846731,9.161165e+08,0.033887,0.033974,0.033999,298.999532
std,672.376156,675.765828,668.683436,672.449177,672.449177,1.543571e+09,0.960907,0.294008,0.166424,163.781890
min,16.660000,16.660000,16.660000,16.660000,16.660000,6.800000e+05,-20.466931,-3.507696,-1.167541,0.091128
25%,85.804999,86.477502,85.097498,85.817499,85.817499,8.870000e+06,-0.402983,-0.122081,-0.054962,175.081223
50%,165.870002,166.664993,164.990005,165.925003,165.925003,9.194500e+07,0.046444,0.055849,0.049736,255.858571
75%,1092.102478,1098.729981,1082.324981,1092.025025,1092.025025,1.180640e+09,0.495540,0.205825,0.137131,471.932210
max,2936.760010,2940.909912,2927.110107,2930.750000,2930.750000,1.145623e+10,11.580037,2.020674,0.839927,597.170130


In [284]:
snp500d['Daily Return'] = snp500d['Adj Close'].pct_change()*100
snp500d['Cumulative Returns'] = snp500d['Daily Return'].cumsum()
snp500d.count()

Date                  17336
Open                  17336
High                  17336
Low                   17336
Close                 17336
Adj Close             17336
Volume                17336
Daily Return          17335
Cumulative Returns    17335
dtype: int64

In [299]:
# snp500d = pd.read_csv('SP500_GSPC_daily.csv')
indexes = snp500d.columns
indexes
new = pd.DataFrame( columns=indexes )
new = new.append( snp500d.iloc[1:5, :] )
snp500d.iloc[ 2:4,:]

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,Cumulative Returns,10DMA,30DMA
2,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000,0.474777,1.615234,NaN,NaN
3,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000,0.295334,1.910567,NaN,NaN


In [296]:
snp500d['10DMA'] = snp500d['Daily Return'].rolling(window=10).mean()
snp500d['30DMA'] = snp500d['Daily Return'].rolling(window=30).mean()
snp500d.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,Cumulative Returns,10DMA,30DMA
0,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000,NaN,NaN,NaN,NaN
1,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000,1.140456,1.140456,NaN,NaN
2,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000,0.474777,1.615234,NaN,NaN
3,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000,0.295334,1.910567,NaN,NaN
4,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000,0.588928,2.499496,NaN,NaN
